# Reconnaissance de couleur par réseau de neurones

In [ ]:
"""
TODO regrouper dans le notebook le code de génération et l'énoncé.

TODO création du dataset à la place de faire un csv externe pour le réimporter derrière
- avec tensorflow_dataset ?
- avec pandas ?
- avec un tableau tout bête (numpy ?) ?

opinion superficielle : avec pandas c'est facile

génération de données avec pandas :
https://towardsdatascience.com/generating-fake-data-with-pandas-very-quickly-b99467d4c618

utilisation de dataset pandas avec tensorflow :
https://www.tensorflow.org/tutorials/load_data/pandas_dataframe

génération de données avec tensorlfow_dataset :
https://www.tensorflow.org/datasets/add_dataset#dataset_example


Liens utiles

vue d'ensemble tensorflow :
https://www.tensorflow.org/tutorials/quickstart/beginner#build_a_machine_learning_model

exemple d'utilisation complète et simple (utilise un tableau pour les données):
https://machinelearningmastery.com/tutorial-first-neural-network-python-keras/

méthode de séparation des données sur un dataset existant :
https://stackoverflow.com/a/50185329/17987233
"""

"\nTODO création du dataset à la place de faire un csv externe pour le réimporter derrière\n- avec tensorflow_dataset ?\n- avec pandas ?\n- avec un tableau tout bête (numpy ?) ?\n\ngénération de données avec pandas :\nhttps://towardsdatascience.com/generating-fake-data-with-pandas-very-quickly-b99467d4c618\n\nutilisation de dataset pandas avec tensorflow :\nhttps://www.tensorflow.org/tutorials/load_data/pandas_dataframe\n\n\nvue d'ensemble tensorflow :\nhttps://www.tensorflow.org/tutorials/quickstart/beginner#build_a_machine_learning_model\n\nexemple d'utilisation complète et simple (utilise un tableau pour les données):\nhttps://machinelearningmastery.com/tutorial-first-neural-network-python-keras/\n\nméthode de séparation des données sur un dataset existant :\nhttps://stackoverflow.com/a/50185329/17987233\n\ngénération de données avec tensorlfow_dataset :\nhttps://www.tensorflow.org/datasets/add_dataset#dataset_example\n"

## Génération du dataset

Exécutez cette cellule sans la modifier pour générez votre dataset, qui sera stocké dans la variable `ds`.

In [ ]:
import csv
import random
import sys
import math

# import tensorflow_datasets as tfds
import pandas as pd


COLORS = {
    "black"     : (0,0,0),
    "silver"    : (192,192,192),
    "gray"      : (128,128,128),
    "white"     : (255,255,255),
    "maroon"    : (128,0,0),
    "red"       : (255,0,0),
    "purple"    : (128,0,128),
    "fuchsia"   : (255,0,255),
    "green"     : (0,128,0),
    "lime"      : (0,255,0),
    "olive"     : (128,128,0),
    "yellow"    : (255,255,0),
    "navy"      : (0,0,128),
    "blue"      : (0,0,255),
    "teal"      : (0,128,128),
    "aqua"      : (0,255,255)
}


def random_rgb():
    """
    Génère un tuple RGB.
    """
    return (
        random.randint(0, 255), 
        random.randint(0, 255), 
        random.randint(0, 255)
        )


def convert(rgb):
    """
    Convertie un tuple rgb qui a pour valeur 0 à 255 vers des valeurs entre 0 et 1.
    """
    r, g, b = rgb
    return r / 255, g / 255, b / 255


def closest_color(rgb):
    """
    On détermine à partir d'un couleur en RGB son nom associé
    (ici on réalise exactement l'opération que le réseau de neurone sera amené à effectuer plus 
    tard).
    """
    r, g, b = rgb
    color_diffs = []
    for color_name in COLORS:
        cr, cg, cb = COLORS[color_name]
        color_diff = math.sqrt((r - cr)**2 + (g - cg)**2 + (b - cb)**2)
        color_diffs.append((color_diff, COLORS[color_name], color_name))
    return min(color_diffs)[2]


# Génère un fichier nommé Dataset.csv
# Il contient 4 colonnes: R, G, B, ColorName

# with open('Dataset.csv', 'w', newline='') as file_:
#     csv_file = csv.writer(file_, delimiter=',')

#     print("Create a dataset of: " + sys.argv[1] + " called Dataset.csv")
#     for _ in range(int(sys.argv[1])):
#         rgb_color = random_rgb()
#         r, g, b = rgb_color
#         rgb_converted = convert(rgb_color)
#         rc, gc, bc = rgb_converted
#         csv_file.writerow([rc, gc, bc, closest_color((r, g, b))])


## Préparation des données

Ramenez les valeurs du dataset entre 0 et 1.  
Séparez vos variables explicatives et à expliquer `X` et `y`.  
Séparez le dataset en lots d'entrainement et de test, pour vous retrouver avec les variables `X_train`, `y_train`, `X_test`, `y_test`.

In [ ]:
# TODO dataset preprocess

## Création du modèle

Créez un modèle possédant le nombre d'entrées et le nombre de sorties adéquat.

Vous devrez choisir :
- le nombre de couches internes et de neurones
- la fonction d'activation
- la fonction de perte

On utilisera la fonction d'optimisation "adam".

Servez-vous des fonctions `Sequential()`, `Input()`, `Dense()` pour créer le modèle.  
Une fois le modèle créé, il se compile avec la fonction `compile()`.

In [ ]:
import tensorflow as tf

INPUT_SIZE = 3
OUTPUT_SIZE = len(COLORS)
HIDDEN_LAYER_NUMBER = 3
HIDDEN_LAYER_NODE_NUMBER = 16

model = tf.keras.Sequential()

# input layer
model.add(tf.keras.layers.Input(shape=(INPUT_SIZE,)))

# hidden layers
for _ in range(HIDDEN_LAYER_NUMBER):
    model.add(tf.keras.layers.Dense(HIDDEN_LAYER_NODE_NUMBER, activation='softmax'))

# output layer
model.add(tf.keras.layers.Dense(OUTPUT_SIZE))

# model compilation
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

## Entraînement

Servez-vous de la fonction `fit()` pour entraîner le modèle avec vos données.

Vous devrez choisir :
- le nombre d'epochs
- la taille des lots

In [ ]:
model.fit(X_train, y_train, epochs=10_000, batch_size=10)

## Évaluation

In [ ]:
loss, accuracy = model.evaluate(X_test, y_test)
print(f"accuracy: {accuracy}\nloss: {loss}")

## Prédictions

In [ ]:
# TODO predictions